In [25]:
import streamlit as st
from sqlalchemy import text
from uuid import uuid4
from datetime import datetime

METADATA_DB_URL = "sqlite:///session_metadata.sqlite"

conn = st.connection(
        name="metadata_connector", # 需要给连接起一个名字，但不需要在 secrets.toml 中定义
        type="sql",
        url=METADATA_DB_URL # 使用变量赋值
    )

# 确保元数据表存在
with conn.session as s:
    s.execute(text("""
        CREATE TABLE IF NOT EXISTS threads_metadata (
            thread_id TEXT PRIMARY KEY,
            name TEXT,
            created_at TEXT
            )
    """))
    s.commit()

# 查询并显示元数据表中的所有线程
df = conn.query(
        "SELECT thread_id, name FROM threads_metadata ORDER BY created_at DESC",
        ttl=60 
    )

thread_id = str(uuid4())  # 生成唯一的thread ID
initial_name = "New Thread"
timestamp = datetime.now().isoformat()

# 插入新的会话信息
with conn.session as s:
        s.execute(
            text("INSERT INTO threads_metadata (thread_id, name, created_at) VALUES (:id, :name, :time)"),
            params={"id": thread_id, "name": initial_name, "time": timestamp}
        )
        s.commit()
conn.reset()


# 更新现有会话的名称
new_name = "Updated Thread Name"
with conn.session as s:
        s.execute(
            text("UPDATE threads_metadata SET name = :new_name WHERE thread_id = :id"),
            params={"new_name": new_name, "id": thread_id}
        )
        s.commit()
conn.reset()



2025-11-26 11:30:12.020 No runtime found, using MemoryCacheStorageManager
2025-11-26 11:30:12.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 11:30:12.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 11:30:12.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 11:30:12.022 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 11:30:12.024 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 11:30:12.024 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 11:30:12.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
# 从langgraph checkpoint读取thread id的消息
